You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [7]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [8]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [9]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [10]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [11]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [12]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [13]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [14]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [15]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [16]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [17]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan Document

Title: Unleashing the Power of Artificial Intelligence: Latest Trends and Key Players

Introduction:
- Brief overview of Artificial Intelligence (AI) and its impact on various industries
- Importance of staying updated on the latest trends and key players in the AI space

Key Points:
1. Latest Trends in Artificial Intelligence:
- Emphasis on advancements in machine learning algorithms
- Growing popularity of natural language processing (NLP) technologies
- Impact of AI on healthcare, fi

I now can give a great answer

Final Answer: 

# Unleashing the Power of Artificial Intelligence: Latest Trends and Key Players

## Introduction

Artificial Intelligence (AI) has become a transformative force across various industries, revolutionizing processes and enhancing capabilities. From predictive analytics in healthcare to personalized recommendations in e-commerce, AI is reshaping the way businesses operate. Staying updated on the latest trends and key players in the AI space is crucial for organizations looking to leverage this cutting-edge technology effectively.

## Latest Trends in Artificial Intelligence

Advancements in machine learning algorithms are at the forefront of AI innovation, enabling more accurate predictions and personalized experiences. The growing popularity of natural language processing (NLP) technologies has also opened up new possibilities for communication between humans and machines. In sectors like healthcare, finance, and retail, AI is driving effic

- Display the results of your execution as markdown in the notebook.

In [18]:
from IPython.display import Markdown
Markdown(result)

# Unleashing the Power of Artificial Intelligence: Latest Trends and Key Players

## Introduction

Artificial Intelligence (AI) has become a transformative force across various industries, revolutionizing processes and enhancing capabilities. From predictive analytics in healthcare to personalized recommendations in e-commerce, AI is reshaping the way businesses operate. Staying updated on the latest trends and key players in the AI space is crucial for organizations looking to leverage this cutting-edge technology effectively.

## Latest Trends in Artificial Intelligence

Advancements in machine learning algorithms are at the forefront of AI innovation, enabling more accurate predictions and personalized experiences. The growing popularity of natural language processing (NLP) technologies has also opened up new possibilities for communication between humans and machines. In sectors like healthcare, finance, and retail, AI is driving efficiency and improving outcomes through automation and data-driven insights.

## Key Players in the AI Industry

Major companies like Google, Amazon, and IBM are leading the charge in the AI revolution, investing heavily in research and development to push the boundaries of what AI can achieve. At the same time, emerging startups are disrupting the market with innovative solutions and agile approaches. Collaboration and partnerships are essential in the AI ecosystem, as companies work together to create synergies and drive innovation at scale.

## Noteworthy News in Artificial Intelligence

Recent developments in AI ethics and regulations highlight the need for responsible AI deployment and governance. Case studies showcasing successful AI implementations demonstrate the tangible benefits of adopting AI technologies. Looking ahead, predictions for the future of AI technology point towards increased automation, enhanced decision-making capabilities, and continued growth across diverse industries.

In conclusion, Artificial Intelligence continues to evolve rapidly, shaping the way businesses operate and interact with customers. By staying informed on the latest trends and key players in the AI space, organizations can position themselves for success in a data-driven world. Subscribe to our newsletter for regular updates on AI trends and key players, attend virtual conferences or webinars on AI technology, and access our free AI resource guide for further learning. Let's unlock the full potential of AI together.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [21]:
topic = "Robotics"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Robotics.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: 

Title: Exploring the Latest Trends in Robotics

Outline:

I. Introduction
- Definition of Robotics
- Importance of Robotics in various industries
- Brief overview of the evolution of Robotics

II. Latest Trends in Robotics
A. Artificial Intelligence integration in Robotics
B. Collaborative Robots (Cobots)
C. Robotics in healthcare industry
D. Autonomous Vehicles
E. Robotics in Agriculture

III. Key Players in Robotics
A. Boston Dynamics
B. ABB Robotics
C. Universal Robots
D. Fanuc Corporation
E. KUKA Roboti

I now can give a great answer

Final Answer: 

# Exploring the Latest Trends in Robotics

## Introduction

Robotics, the interdisciplinary branch of engineering and science that involves the design, construction, operation, and use of robots, has become an integral part of various industries. From manufacturing to healthcare, robotics plays a crucial role in improving efficiency, accuracy, and safety. The evolution of robotics has seen significant advancements over the years, with the integration of artificial intelligence and the rise of collaborative robots (Cobots) changing the landscape of automation.

## Latest Trends in Robotics

### Artificial Intelligence integration in Robotics

One of the most significant trends in robotics is the integration of artificial intelligence (AI) to enhance the capabilities of robots. AI allows robots to analyze data, learn from their experiences, and make decisions without human intervention. This trend has led to the development of more advanced 

In [22]:
Markdown(result)

# Exploring the Latest Trends in Robotics

## Introduction

Robotics, the interdisciplinary branch of engineering and science that involves the design, construction, operation, and use of robots, has become an integral part of various industries. From manufacturing to healthcare, robotics plays a crucial role in improving efficiency, accuracy, and safety. The evolution of robotics has seen significant advancements over the years, with the integration of artificial intelligence and the rise of collaborative robots (Cobots) changing the landscape of automation.

## Latest Trends in Robotics

### Artificial Intelligence integration in Robotics

One of the most significant trends in robotics is the integration of artificial intelligence (AI) to enhance the capabilities of robots. AI allows robots to analyze data, learn from their experiences, and make decisions without human intervention. This trend has led to the development of more advanced and intelligent robots that can perform complex tasks with precision and accuracy.

### Collaborative Robots (Cobots)

Collaborative robots, or Cobots, are designed to work alongside humans in a shared workspace. Unlike traditional industrial robots that are isolated in cages, Cobots are equipped with sensors and advanced safety features that allow them to collaborate with human workers safely. This trend is revolutionizing the manufacturing industry by improving productivity and flexibility in production processes.

### Robotics in healthcare industry

The healthcare industry is increasingly adopting robotics to improve patient care and streamline medical procedures. Robots are used in surgical operations, patient care, and rehabilitation to enhance precision, reduce human error, and increase efficiency. The integration of robotics in healthcare is helping medical professionals deliver better outcomes for patients and revolutionizing the way healthcare services are delivered.

### Autonomous Vehicles

Autonomous vehicles, also known as self-driving cars, are another major trend in robotics. These vehicles use sensors, cameras, and artificial intelligence to navigate roads and transport passengers without human intervention. The development of autonomous vehicles has the potential to revolutionize transportation systems, reduce accidents, and improve overall road safety.

### Robotics in Agriculture

Robotics is also making significant strides in the agriculture industry, with robots being used for planting, harvesting, and monitoring crops. Agricultural robots are equipped with sensors and cameras that allow them to analyze soil conditions, detect pests, and optimize crop yield. This trend is transforming the way farmers manage their operations, increasing efficiency and sustainability in agriculture.

## Key Players in Robotics

### Boston Dynamics

Boston Dynamics is a leading robotics company known for its innovative and agile robots, including the famous humanoid robot, Atlas. The company focuses on developing advanced robots for various industries, such as logistics, construction, and defense.

### ABB Robotics

ABB Robotics is a global leader in industrial robotics, offering a wide range of robotic solutions for manufacturing, automotive, and electronics industries. The company's robots are known for their precision, reliability, and efficiency in production processes.

### Universal Robots

Universal Robots specializes in collaborative robots that are designed to work alongside humans in a shared workspace. The company's robots are easy to program, flexible, and safe, making them ideal for small and medium-sized enterprises.

### Fanuc Corporation

Fanuc Corporation is a Japanese robotics company that produces industrial robots and automation systems for a wide range of industries, including automotive, electronics, and food and beverage. The company's robots are known for their high-speed performance and reliability in manufacturing processes.

### KUKA Robotics

KUKA Robotics is a German robotics company that specializes in robotic arms and automation systems for industries such as automotive, aerospace, and logistics. The company's robots are known for their precision, agility, and versatility in various applications.

## Noteworthy News in Robotics

### Recent advancements in Robotics technology

The field of robotics is constantly evolving, with new advancements and innovations shaping the future of automation. From humanoid robots to swarm robotics, the latest developments in robotics technology are pushing the boundaries of what is possible with robots.

### Impact of Robotics on the job market

The rise of robotics and automation has raised concerns about the impact on the job market. While robots can perform repetitive and dangerous tasks more efficiently than humans, there are growing fears about job displacement and the need for reskilling the workforce to adapt to the changing labor market.

### Ethical considerations in Robotics development

As robotics technology becomes more advanced, ethical considerations around the use of robots are becoming increasingly important. Questions about robot rights, privacy, and accountability are being raised as robots become more integrated into society.

## Target Audience Analysis

Robotics enthusiasts, industry professionals, and students and educators in STEM fields are the target audience for this blog post. Whether you are passionate about the latest trends in robotics, working in the industry, or studying robotics in school, this post aims to provide valuable insights and updates on the evolving field of robotics.

## SEO Keywords

- Robotics trends 2021
- Top Robotics companies
- Robotics news updates
- Robotics technology advancements

## Resources

- Robotics Online Magazine
- Robotics Business Review
- IEEE Robotics and Automation Society

### Call to Action

Stay informed about the latest trends in Robotics by subscribing to our newsletter for regular updates and insights on Robotics technology. 

In conclusion, robotics continues to be at the forefront of technological innovation, with AI integration, collaborative robots, and advancements in various industries driving the evolution of automation. As we look towards the future, it is essential to stay informed about the latest trends and developments in robotics to understand the impact on society, the economy, and the workforce. Robotics is not just about machines; it is about reshaping the way we live and work in a rapidly changing world.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).